Accessing grib data

In [ ]:
import os
from pathlib import Path
from typing import Iterable
from datetime import datetime
from shutil import copyfileobj

from requests import Session, HTTPError
import pandas as pd


In [ ]:
def google_api_hrrr_grib2_data(
    start:datetime=datetime(2022,3,1), 
    end:datetime=datetime(2022,6,1)
    )->Iterable[tuple[str,str]]:
    """
    generate a date range on an hourly interval
    """
    
    base_url ="https://storage.googleapis.com/high-resolution-rapid-refresh/"
    date_range = pd.date_range(start, end, freq="h")

    return zip(
        date_range.strftime("%Y-%m-%dT%H:%M:%SZ"),
        base_url + date_range.strftime("hrrr.%Y%m%d/conus/hrrr.t%Hz.wrfnatf00.grib2")
        )

In [ ]:

def download_and_gribs(path :Path)->list[str]:
    """
    iterate over validtimes and urls and save files to a Path directory
    returns a list of urls that failed to download
    """
    failed_files = []
    # request context manager 
    with Session() as session:
        # iteratate over the generator function
        for validtime, url in google_api_hrrr_grib2_data():
            # add the filename to the path object
            save_to = path / f"{validtime}.grib" 
            try:
                # make a http get request to the url
                res = session.get(url, stream=True)
                # on non 200 status code raise HTTPError
                res.raise_for_status()
                # save the file to the directory
                with save_to.open("wb") as fileout:
                    copyfileobj(res.raw, fileout)
                print("grib2 file saved at ",save_to)

            except HTTPError:
                # on a failed http reqest append the url to the failed_files list
                failed_files.append(url)
                
    return failed_files


media_ext = Path(
    os.getenv("EXTERNAL_DRIVE","/media/external/hrrr/conus/")
    ) 

download_and_gribs(media_ext)